In [1]:
from Ori_data import *

import numpy as np
import pandas as pd
import scanpy as sc
import h5py

#import scripro

import anndata
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import scanpy as sc
import warnings
warnings.filterwarnings("ignore")

In [3]:
sc._settings.ScanpyConfig.n_jobs = 4

In [4]:
rna = sc.read_h5ad('/fs/home/xuyunfan/data/ORF/select_tf.h5ad')

In [5]:
sc.tl.leiden(rna)

In [6]:
rna = rna[rna.obs[rna.obs.leiden.isin(rna.obs.leiden.value_counts()[rna.obs.leiden.value_counts() > 100].index)].index]

In [7]:
rna.var_names_make_unique()

In [ ]:
sc.tl.rank_genes_groups(rna,'leiden', method='t-test',n_jobs=8)

In [ ]:
test_data = Ori_Data(rna,Cell_num=50)

Compute supercell: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 72/72 [31:10<00:00, 25.98s/it]


In [5]:
adata_raw = rna.raw.to_adata()

In [6]:
cluster_method='leiden'

In [22]:
adList = []
adList_obs = {}

subcluster_adata = {}

for i in set(rna.obs[cluster_method]):
    leiden_index = rna.obs.loc[rna.obs[cluster_method] == i].index
    sub_cluster = adata_raw[leiden_index]
    subcluster_adata[i] = sub_cluster

def process_sub_cluster(i, cell_num):
    merged_data, obs = supercell_pipeline(i[1], cell_num=cell_num, verbose=False)
    merged_data_index = [i[0] + "_" + str(j) for j in range(0, merged_data.shape[0])]
    merged_data.obs = pd.DataFrame(index=merged_data_index)
    return merged_data, i[0], obs

In [23]:
with ProcessPoolExecutor(max_workers=2) as executor:
    futures = [executor.submit(process_sub_cluster,i,50) for i in subcluster_adata.items()]
    for future in tqdm(as_completed(futures),total=len(futures),desc="Compute supercell"):
        merged_data, i, obs = future.result()
        adList.append(merged_data)
        adList_obs[i] = obs

Processing markers:  35%|█████████████████████████████████████████████████████▍                                                                                                    | 25/72 [09:27<17:46, 22.69s/it]Process ForkProcess-3:
Traceback (most recent call last):
  File "/fs/home/xuyunfan/miniconda3/envs/scripro_env/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/fs/home/xuyunfan/miniconda3/envs/scripro_env/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/fs/home/xuyunfan/miniconda3/envs/scripro_env/lib/python3.8/concurrent/futures/process.py", line 233, in _process_worker
    call_item = call_queue.get(block=True)
  File "/fs/home/xuyunfan/miniconda3/envs/scripro_env/lib/python3.8/multiprocessing/queues.py", line 96, in get
    with self._rlock:
  File "/fs/home/xuyunfan/miniconda3/envs/scripro_env/lib/python3.8/multiprocessing/synchronize.py", line 95, in __enter__
    re

In [14]:
ad_all = ad.concat(adList, join='outer')

In [16]:
ad_all.to_df()

,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2MP1,A3GALT2,A4GALT,A4GNT,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
64_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,2.0
64_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,2.0
64_2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,2.0,1.0,0.0,2.0
64_3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,2.0
64_4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6_31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,2.0
6_32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,2.0
6_33,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,2.0
6_34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,2.0


In [10]:
adList_obs['64']

,TF,batch,louvain,n_counts,n_genes,percent_mito,dpt_pseudotime,v_pseudotime,first_part,leiden
"R1.57,R2.51,R3.13,P1.48-3-4-0",TFORF0799-ATF3,0,1,2831.0,1995,0.020134,0.011033,0.011321,TFORF0799,2
"R1.83,R2.02,R3.61,P1.07-2-1-0",TFORF0801-ATF3,0,2,1354.0,1119,0.028065,0.013189,0.027285,TFORF0801,3
"R1.04,R2.24,R3.48,P1.31-1-2-0",TFORF0801-ATF3,0,1,5044.0,3057,0.027161,0.010886,0.006979,TFORF0801,6
"R1.56,R2.37,R3.84,P1.23-0-2-0",TFORF0699-BATF2,0,0,4799.0,2880,0.030840,0.011286,0.019576,TFORF0699,0
"R1.10,R2.66,R3.61,P1.40-2-4-0",TFORF0698-BATF2,0,1,3822.0,2526,0.023548,0.011220,0.001125,TFORF0698,3
...,...,...,...,...,...,...,...,...,...,...
"R1.57,R2.27,R3.11,P1.47-3-2-0",TFORF1383-ZNF788,0,0,5487.0,3256,0.019683,0.013559,0.016236,TFORF1383,0
"R1.22,R2.39,R3.43,P1.47-3-2-0",TFORF1384-ZNF788,0,2,1429.0,1169,0.020294,0.012364,0.014814,TFORF1384,6
"R1.53,R2.95,R3.48,P1.30-0-0",TFORF1384-ZNF788,0,2,1520.0,1240,0.018421,0.011297,0.004388,TFORF1384,2
"R1.75,R2.10,R3.18,P1.16-3-3-0",TFORF1383-ZNF788,0,2,1528.0,1214,0.036649,0.014052,0.006971,TFORF1383,5


In [19]:
rna.obs['new_leiden'] = np.zeros(len(rna.obs))

In [20]:
for i in adList_obs.keys():
    for k in adList_obs[i].index:
        rna.obs.loc[k,'new_leiden'] = i+"_"+adList_obs[i].loc[k,'leiden']  

In [21]:
rna.obs

,TF,batch,louvain,n_counts,n_genes,percent_mito,dpt_pseudotime,v_pseudotime,first_part,leiden,new_leiden
"R1.22,R2.92,R3.04,P1.40-2-4-0",TFORF0800-ATF3,0,8,3209.0,2179,0.032097,0.016058,0.007092,TFORF0800,65,65_5
"R1.12,R2.78,R3.95,P1.16-3-3-0",TFORF0801-ATF3,0,4,1750.0,1274,0.008000,0.036652,0.219516,TFORF0801,2,2_48
"R1.30,R2.56,R3.85,P1.30-0-0",TFORF0798-ATF3,0,4,2475.0,1816,0.019394,0.027471,0.052197,TFORF0798,6,6_17
"R1.57,R2.57,R3.28,P1.62-1-1-0",TFORF0801-ATF3,0,1,3110.0,2136,0.025080,0.010836,0.004030,TFORF0801,4,4_1
"R1.13,R2.80,R3.13,P1.63-1-3-0",TFORF0801-ATF3,0,1,6779.0,3729,0.026700,0.011129,0.008793,TFORF0801,7,7_14
...,...,...,...,...,...,...,...,...,...,...,...
"R1.02,R2.50,R3.42,P1.62-1-1-0",TFORF0494-ZSCAN2,0,1,6066.0,3409,0.026212,0.011011,0.007089,TFORF0494,45,45_1
"R1.42,R2.78,R3.50,P1.16-3-3-0",TFORF0494-ZSCAN2,0,0,2143.0,1566,0.030798,0.013749,0.006548,TFORF0494,0,0_32
"R1.88,R2.12,R3.12,P1.22-0-0",TFORF0494-ZSCAN2,0,5,2901.0,2042,0.021027,0.011470,0.002121,TFORF0494,10,10_19
"R1.78,R2.62,R3.20,P1.16-3-3-0",TFORF0494-ZSCAN2,0,0,7011.0,3774,0.026244,0.011237,0.013153,TFORF0494,1,1_54
